<a href="https://colab.research.google.com/github/ijazahmad-star/Colab-Workspace/blob/main/Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community langchain_text_splitters langchain_huggingface langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.6 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.7.0 which is incompatible.


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage
from langchain_community.vectorstores import FAISS
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_community.agent_toolkits.load_tools import load_tools
from langchain_community.tools import DuckDuckGoSearchRun
import os

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
model  = HuggingFaceEmbeddings(model_name="ibm-granite/granite-embedding-english-r2")

In [ ]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 21.5 MB/s eta 0:00:00


In [ ]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 67.5 MB/s eta 0:00:00


In [ ]:
loader = PyPDFLoader("/content/p2.pdf")
pages = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1200,
    chunk_overlap = 300,
)

chunks = splitter.split_documents(pages)
# print(chunks[4].metadata)

vectorstore = FAISS.from_documents(chunks, model)

In [ ]:
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai", api_key = api_key)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=retriever, chain_type="stuff",
)

In [ ]:
pip install -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 115.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 111.0 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


In [ ]:
## main cell
web_search_tool = Tool(
    name="web_search_tool",
    func=DuckDuckGoSearchRun().run,
    description="Search the internet using DuckDuckGo for latest info."
)

@tool
def rag_tool(query: str) -> str:
    """This function will help to retrieve information from the pdf"""
    result = qa_chain.invoke({"query": query})
    return result.get("result", str(result))

tools = [rag_tool, web_search_tool]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
while True:
    query = input("\nYou: ")
    if query.lower() in ["exit", "quit"]:
        print("Exiting RAG Agent.")
        break
    else:
        messages = [HumanMessage(query)]
        ai_msg = llm_with_tools.invoke(messages)
        messages.append(ai_msg)
        for tool_call in ai_msg.tool_calls:
            selected_tool = {"rag_tool": rag_tool, "web_search_tool": web_search_tool}[tool_call["name"].lower()]
            tool_msg = selected_tool.invoke(tool_call)
            messages.append(tool_msg)

        final_msg = llm_with_tools.invoke(messages)
        print("AI Agent:", final_msg.content)



You: summarize this document
AI Agent: This document, titled "Understanding Domestic Violence," was compiled by Zohra Ahmed for The Jugnu Project and focuses specifically on domestic violence in Pakistan. It draws on a wide range of recent sources (mostly 2021-2024), including research on domestic violence, human rights reports (such as those from Human Rights Watch and NCHR Pakistan), data from the Pakistan Demographic and Health Survey, reports on child abuse, legal frameworks for domestic violence in Pakistan, global studies on homicide and gender-based killing, and reports on gender parity and women's economic participation in Pakistan.

You: what is the result of pak vs ind war in 2025?
AI Agent:  I can provide historical information if you'd like.

You: current caption of pakistan men team
AI Agent: Mohammad Rizwan has been appointed as the new white-ball captain of the Pakistan men's cricket team. He will lead the team in the upcoming tours of Australia and Zimbabwe, with his f